In [178]:
from sklearn import svm
from sklearn import metrics
import pandas as pd
import random
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


## Import Data and split 

In [179]:
df=pd.read_csv("spam.data",header=None,delimiter=r"\s+")
df=shuffle(df)
name=df.values
x=df[df.columns[:-1]].values
y=df[df.columns[-1]].values
avg=pd.read_csv("threshold.csv",delimiter=r"\s+",header=None)
avg=avg.values.tolist()
f = []       
feat = [[name[i][j] for i in range(len(name))] for j in range(len(name[0]))]
avg2 = []
for i in avg:
    avg2.append(i[0])
for i in range(len(feat)):
     f.append(list(map(lambda x : 0 if float(x) < avg2[i] else 1, feat[i])))

f = np.transpose(f)
a=f[:,:56]
b=f[:,57]

X_train, X_test, y_train, y_test = train_test_split(a, b, test_size = 0.3)

## Class Decision Tree containing all functions to run random forest

#### Accuracy :  0.7074583635047067  
#### CPU times: user 50.2 s, sys: 84 ms, total: 50.3 s  
#### Wall time: 50.3 s  

In [188]:
%%time
class DecisionTree:
    def __init__(self):
        import csv
        import numpy
        import math
        self.csv = csv
        self.np = numpy
        self.math = math
        self.n = Node() 
    def entropy(self, l):
        entropy = 0
        for i in range(len(l)):
            total = l[i][0] + l[i][1]
            prob1 = l[i][0]
            prob2 = l[i][1]
            if(prob1 == 0 or prob2 == 0):
                entropy += 0
            else:
                p1 = float(prob1)/total
                p2 = float(prob2)/total
                entropy += (total*(-p1*np.log2(p1) - p2*np.log2(p2)))
        return entropy
    def random_forest(self,m,n_trees,X_train,X_test):
        prediction=[]
        for i in range(n_trees): 
            att = []
            count1 = 0
            while(count1 != m):
                p = random.randrange(0,len(X_train[0])-1)
                if(p not in att):
                    att.append(p)
                    count1 += 1
            att.append(len(X_train[0])-1)
            f_1 = X_test[:,[x for x in att]]
            f_2 = X_train[:,[x for x in att]]
            self.train(f_2)
            pred = self.predict(f_1)
            prediction.append(pred)

        temp1 = np.transpose(np.array(prediction))
        temp1.tolist()
        final_pred = []

        for i in range(len(temp1)):
            c = Counter(temp1[i])
            value,count= c.most_common()[0]
            # print(value,count)
            final_pred.append(value)


        ascore=0
        for i in range(len(y_test)):
            if(final_pred[i] == y_test[i]):
                ascore+=1

        accuracy = float(ascore)/len(final_pred)
        print("Accuracy : " ,accuracy)
        #print("Accuracy:",metrics.accuracy_score(y_test, temp1))
    def predict(self, data):
        f = [self.pr(i) for i in data]
        return f
    def modify(self, dataset, newf, l, data):
        #Dataset at a node is divided on among its child nodes
        for i in dataset:
            x = data[i][newf]
            y = data[i][len(data[0])-1]
            l[x].append(i)
            l[x][y] += 1
    def pr(self, l):
        # Given test data, go to the root of the decision tree. Compare splitting feature with corresponding value
        # Go to appropriate child node. Keep on doing this until a leaf node is encountered
        temp = self.n
        feat = temp.feature
        while feat!=-1:
            temp = temp.children[(l[feat])]
            feat = temp.feature
        # If leaf has no entry, go to parent node.
        while temp.yes + temp.no == 0:
            temp = temp.parent
        # If no of yes is more, classify in class 1. Else in class 0
        if temp.yes > temp.no:
            return 1
        else:
            return 0
    def run(self, node, unassign, datasets, feature, data):
        unassigned = unassign[:]
        dataset = datasets[:]
        for i in dataset:
            if data[i][len(data[0])-1] == 1:
                node.yes += 1
            else:
                node.no += 1
        # Breaking if no more dataset/feature to split or entropy is low
        if unassigned == [] or dataset == []:
            return
        x = unassigned[0]
        l = [[0, 0] for _ in range(feature[x])]
        self.modify(dataset, x, l, data)
        et = self.entropy(l)
        
        m = []
        for i in unassigned[1:]:
            m = [[0, 0] for _ in range(feature[i])]
            self.modify(dataset, i, m, data) 
            temp = self.entropy(m)      
            if(temp < et):
                et = temp
                x = i
                l = m

        node.feature = x
        unassigned.remove(x)

        # Creating children node. # = no of values splitting feature can take
        for i in range(len(l)):
            child = Node()
            child.parent = node
            node.add_child(child)
        # Recursively calling the function
        for i in range(len(l)):
            self.run(node.children[i], unassigned, l[i][2:],feature,data)
    def train(self, data):
        unassign = [x for x in range(len(data[0])-1)]
        dataset = [x for x in range(len(data))]
        feature = [2 for i in range(len(data[0])-1)]
        self.run(self.n, unassign, dataset, feature, data)
def main():
    m=31
    n_trees = 19
    model = DecisionTree()
    model.random_forest(m,n_trees,X_train,X_test)
main()

Accuracy :  0.7074583635047067
CPU times: user 50.2 s, sys: 84 ms, total: 50.3 s
Wall time: 50.3 s


In [157]:
class Node(object):
    def __init__(self):
        self.feature = -1
        self.children = []
        self.yes = 0
        self.no = 0
        self.parent = -1
    def add_child(self, obj):
        self.children.append(obj)


#### Accuracy: 0.943519188993483  
#### CPU times: user 252 ms, sys: 0 ns, total: 252 ms  
#### Wall time: 251 ms  


In [187]:

from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(bootstrap=True,n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.939174511223751
Accuracy: 0.942795076031861
Accuracy: 0.942070963070239
Accuracy: 0.944243301955105
Accuracy: 0.941346850108617
Accuracy: 0.939174511223751
Accuracy: 0.941346850108617
Accuracy: 0.942070963070239
252 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
